In [ ]:
############################################################################
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
############################################################################
#  DISCLAIMER
# The code shared by CEs with customers must be limited to non-production
# demo/example code or infrastructure automation scripts.
############################################################################
## Colab: @martincabrera , @scavaletto (en los monitos)

# Vertex AI SDK for Python: applied examples **chat-bison model and save memory chat**
<table align="left">
  <td>
    <a href=" ">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href=" ">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
<a href=" " target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Installing the package

In [ ]:
# We have to first authenticate
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
!pip install google-cloud-aiplatform
!pip install google-api-python-client
!pip install langchain
!pip install google-search-results

\! ^^^^ Do not forget to click the "Restart runtime" button above.


## Authenticating

In [ ]:
PROJECT_ID = "cloud-llm-preview1"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

SERPAPI_API_KEY= "YOUR KEY SERPAPI"  # @param {type:"string"}
# Esta api se obtiene de: https://serpapi.com/

# Librerias Langchain + VertexAI

In [ ]:
import os
import vertexai
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent

In [ ]:
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# To get your Google API Key, go to GCP Console/APIs & Services/Credentials
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY

## Chatbot code

In [ ]:
from aiohttp import client
import json

class ClientInformationTool(object):

    def run(self, message):
        print('Tool: ' + message)
        datos_cliente = {
            "nombre": "Stefanni Nicole",
            "apellido": "Cavaletto Hevia",
            "direccion": "Las siemprevivas #1111, La Flower, Santiago",
            "telefono": "+569 65929837",
            "renta_actual" : {
                "monto" : 850000,
                "fecha_actualizacion" : "2022-11-30"
            },
            "cuenta_ahorro" : {
                "numero_cuenta": "123456789",
                "saldo" : 125500
            },
            "creditos" : [
                {
                    "monto" : 1500000.00,
                    "cuotas" : 12,
                    "valor_cuota" : 128125,
                    "ultima_cuota_pagada" : 5,
                    "tasa_interes" : 2.5,
                },
                {
                    "monto" : 850000.00,
                    "cuotas" : 12,
                    "valor_cuota" : 72604,
                    "ultima_cuota_pagada" : 3,
                    "tasa_interes" : 2.5,
                },
            ]
        }

        return json.dumps(datos_cliente)

search = SerpAPIWrapper()
client_information = ClientInformationTool()

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="Utiliza esta tool para buscar información en la web sobre asesoramiento financiero"
    ),
    Tool(
        name = "Client Information",
        func=client_information.run,
        description="Utiliza esta tool para consultar la información financiera del cliente"
    ),
]


In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")
llm=VertexAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

## Ejemplos de conversación

In [ ]:
agent_chain.run(input="Hola, soy Stefanni")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Hola Stefanni, ¿cómo puedo ayudarte hoy?

> Finished chain.


'Hola Stefanni, ¿cómo puedo ayudarte hoy?'

In [ ]:
agent_chain.run("Qué productos financieros puedo utilizar para ahorrar dinero mensualmente?")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Hay muchos productos financieros que puedes utilizar para ahorrar dinero mensualmente. Algunos de los más populares son las cuentas de ahorro, los certificados de depósito y los fondos de inversión. Cada uno de estos productos tiene sus propias ventajas y desventajas, por lo que es importante que elijas el que mejor se adapte a tus necesidades. Si necesitas ayuda para elegir un producto financiero, puedes consultar con un asesor financiero.

> Finished chain.


'Hay muchos productos financieros que puedes utilizar para ahorrar dinero mensualmente. Algunos de los más populares son las cuentas de ahorro, los certificados de depósito y los fondos de inversión. Cada uno de estos productos tiene sus propias ventajas y desventajas, por lo que es importante que elijas el que mejor se adapte a tus necesidades. Si necesitas ayuda para elegir un producto financiero, puedes consultar con un asesor financiero.'

In [ ]:
agent_chain.run("Cuál es el saldo de mi cuenta corriente?")



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Client Information
Action Input: Tool: 

Observation: {"nombre": "Stefanni Nicole", "apellido": "Cavaletto Hevia", "direccion": "Las siemprevivas #1111, La Flower, Santiago", "telefono": "+569 65929837", "renta_actual": {"monto": 850000, "fecha_actualizacion": "2022-11-30"}, "cuenta_ahorro": {"numero_cuenta": "123456789", "saldo": 125500}, "creditos": [{"monto": 1500000.0, "cuotas": 12, "valor_cuota": 128125, "ultima_cuota_pagada": 5, "tasa_interes": 2.5}, {"monto": 850000.0, "cuotas": 12, "valor_cuota": 72604, "ultima_cuota_pagada": 3, "tasa_interes": 2.5}]}
Thought:Do I need to use a tool? No
AI: Stefanni, el saldo de tu cuenta corriente es de 0 pesos.

> Finished chain.


'Stefanni, el saldo de tu cuenta corriente es de 0 pesos.'

In [ ]:
agent_chain.run("Cuántos créditos tengo?")



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Client Information
Action Input: Stefanni, cuántos créditos tienes?Tool: Stefanni, cuántos créditos tienes?

Observation: {"nombre": "Stefanni Nicole", "apellido": "Cavaletto Hevia", "direccion": "Las siemprevivas #1111, La Flower, Santiago", "telefono": "+569 65929837", "renta_actual": {"monto": 850000, "fecha_actualizacion": "2022-11-30"}, "cuenta_ahorro": {"numero_cuenta": "123456789", "saldo": 125500}, "creditos": [{"monto": 1500000.0, "cuotas": 12, "valor_cuota": 128125, "ultima_cuota_pagada": 5, "tasa_interes": 2.5}, {"monto": 850000.0, "cuotas": 12, "valor_cuota": 72604, "ultima_cuota_pagada": 3, "tasa_interes": 2.5}]}
Thought:Do I need to use a tool? No
AI: Stefanni, tienes dos créditos: uno de 1500000 pesos con 12 cuotas de 128125 pesos y otro de 850000 pesos con 12 cuotas de 72604 pesos.

> Finished chain.


'Stefanni, tienes dos créditos: uno de 1500000 pesos con 12 cuotas de 128125 pesos y otro de 850000 pesos con 12 cuotas de 72604 pesos.'

In [ ]:
agent_chain.run("todavía recuerdas mi nombre?")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Sí, Stefanni, recuerdo tu nombre.

> Finished chain.


'Sí, Stefanni, recuerdo tu nombre.'